In [4]:
from scipy.optimize._optimize import OptimizeResult as opt
dir(opt)

['__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [1]:
import numpy as np

# example arrays
arr1 = np.array([1, 2, 3, 2, 4, 5, 3, 1, 5, 6])
arr2 = np.array([2, 4, 6])

# create mask
mask = np.in1d(arr1, arr2)

print(mask)


[False  True False  True  True False False False False  True]


In [3]:
arr1 = np.array([1, 2, 3, 2, 4, 5, 3, 1, 5, 6])
mask = np.array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1]).astype(bool)
mask[arr1]


array([False,  True, False,  True,  True, False, False, False, False,
        True])

In [4]:
from scipy.sparse import lil_matrix  # type: ignore
from scipy.sparse.csgraph import reverse_cuthill_mckee  # type: ignore
import numpy as np
import scipy.sparse as sp


In [ ]:
def sfd(x, grad, Hstr, group, dx):
    xcurr = x.flatten()
    m, n = Hstr.shape
    v = np.zeros(n)
    ncol = np.max(group)

    alpha = 1e-7 * np.ones(ncol)

    #H = lil_matrix(Hstr != 0, dtype=float)  # Equivalent of MATLAB's spones
    rowH, colH = Hstr.nonzero()
    all_new_rows = []
    all_new_cols = []
    all_new_data = []
    for k in range(ncol):
        d = (group == k + 1)

        xnrm = max(np.linalg.norm(xcurr[d]), 1)  # type: ignore
        alpha[k] *= xnrm
        y = xcurr + alpha[k] * d

        v = dx(y)
        w = (v - grad) / alpha[k]
        mask_H_cols = d[colH]

        new_rows = rowH[mask_H_cols]
        new_cols = colH[mask_H_cols]
        new_data = w[new_rows]
        all_new_rows.append(new_rows)
        all_new_cols.append(new_cols)
        all_new_data.append(new_data)

    new_rows_singe_array = np.concatenate(all_new_rows + all_new_cols)
    new_cols_singe_array = np.concatenate(all_new_cols + all_new_rows)
    new_data_singe_array = np.concatenate(all_new_data + all_new_data) / 2
    print(np.min(alpha))
    H = sp.csc_matrix((new_data_singe_array, (new_rows_singe_array, new_cols_singe_array)), shape=(m, n))  # type: ignore
    return H


In [5]:
a = [1 , 3 , 5 , 7 , 9]
b = [2 , 4 , 6 , 8 , 10]
a + b

[1, 3, 5, 7, 9, 2, 4, 6, 8, 10]

In [1]:
from scipy.sparse.linalg import factorized
factorized


<function scipy.sparse.linalg._dsolve.linsolve.factorized(A)>

In [7]:
!python -m ensurepip --upgrade

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /tmp/tmpzjr3b90n


In [8]:
!pip install --use-pep517 cyipopt

Defaulting to user installation because normal site-packages is not writeable
  Using cached cyipopt-1.2.0.tar.gz (46 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [60 lines of output]
      <string>:26: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
      /usr/bin/python: No module named pip
      Traceback (most recent call last):
        File "/tmp/pip-build-env-ybwgfhni/overlay/lib/python3.11/site-packages/setupto

In [3]:
import numpy as np
import cyipopt
from scipy.sparse import csc_matrix

n_variables = 10

# Objective function


def eval_f(x, user_data=None):
    return np.sum(x**2)

# Gradient of the objective function


def eval_grad_f(x, user_data=None):
    return 2 * x

# Hessian of the objective function


def eval_h(x, lagrange, obj_factor, user_data=None):
    hessian = obj_factor * 2 * np.eye(n_variables)
    return csc_matrix(hessian)  # Return as sparse matrix

# Hessian Structure of the objective function


def eval_h_structure(n_variables, user_data=None):
    iRow = np.arange(n_variables)
    jCol = np.arange(n_variables)
    return (iRow, jCol)


# Define problem
nlp = cyipopt.problem(
    n=n_variables,
    m=0,
    problem_obj=cyipopt.PyIpoptProblemSparse(
        n=n_variables,
        m=0,
        eval_f=eval_f,
        eval_grad_f=eval_grad_f,
        eval_h=eval_h,
        eval_h_structure=eval_h_structure,
    )
)

# Initial guess
x0 = np.ones(n_variables)

# Set lower and upper bounds
lb = -np.ones(n_variables)
ub = np.ones(n_variables)

nlp.add_bounds(lb, ub)

# Solve problem
x, info = nlp.solve(x0)


ModuleNotFoundError: No module named 'cyipopt'